In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
um_classes = 10
input_shape = (32, 32, 3)

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

print("x_train shape: {} - y_train shape: {}".format(X_train.shape,y_train.shape))
print("x_test shape: {} - y_test shape: {}".format(X_test.shape,y_test.shape))


170508288/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


In [3]:
# Scale images to the [0, 1] range
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, um_classes)
y_test = keras.utils.to_categorical(y_test, um_classes)

In [4]:
# merge the original train and test data together
x_train=np.concatenate((X_train,X_test))
y_train=np.concatenate((y_train, y_test))
print("x_train shape: {} - y_train shape: {}".format(x_train.shape,y_train.shape))

x_train shape: (60000, 32, 32, 3) - y_train shape: (60000, 10)


In [5]:
from sklearn.model_selection import train_test_split
#split test and the remaining data
X_remain, X_test, y_remain, y_test= train_test_split(x_train,y_train,test_size=0.2)
print("x_test shape: {} - y_test shape: {}".format(X_test.shape,X_test.shape))

x_test shape: (12000, 32, 32, 3) - y_test shape: (12000, 32, 32, 3)


In [6]:
X_train, X_discard, y_train, y_discard= train_test_split(X_remain,y_remain,train_size=0.1)

In [7]:
# customized model
inputs = keras.Input(shape=(32, 32, 3))
x=layers.Conv2D(32, kernel_size=(3, 3))(inputs)
x=layers.BatchNormalization()(x)
x=layers.Activation("relu")(x)

#x=layers.Dropout(0.1)(x)

x=layers.MaxPooling2D(pool_size=(2, 2))(x)

x=layers.Conv2D(64, kernel_size=(3, 3))(x)
x=layers.BatchNormalization()(x)
x=layers.Activation("relu")(x)

#x=layers.Dropout(0.1)(x)

x=layers.MaxPooling2D(pool_size=(2, 2))(x)
x=layers.Flatten()(x)
outputs=layers.Dense(um_classes, activation="softmax")(x)

model_customized=keras.Model(inputs,outputs)
model_customized.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model_customized.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 30, 30, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496 

In [8]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
history = model_customized.fit(X_train, y_train, epochs=10, batch_size=32, callbacks=[callback], validation_split=0.2, verbose=1)


Epoch 1/10
120/120 [==============================] - 10s 11ms/step - loss: 1.9830 - accuracy: 0.3372 - val_loss: 2.6398 - val_accuracy: 0.1094
Epoch 2/10
120/120 [==============================] - 1s 9ms/step - loss: 1.5019 - accuracy: 0.4698 - val_loss: 3.3054 - val_accuracy: 0.1469
Epoch 3/10
120/120 [==============================] - 1s 9ms/step - loss: 1.3067 - accuracy: 0.5359 - val_loss: 2.3716 - val_accuracy: 0.2396
Epoch 4/10
120/120 [==============================] - 1s 9ms/step - loss: 1.1003 - accuracy: 0.6195 - val_loss: 1.8732 - val_accuracy: 0.3250
Epoch 5/10
120/120 [==============================] - 1s 9ms/step - loss: 0.9877 - accuracy: 0.6549 - val_loss: 1.5604 - val_accuracy: 0.4458
Epoch 6/10
120/120 [==============================] - 1s 9ms/step - loss: 0.8626 - accuracy: 0.7055 - val_loss: 1.4485 - val_accuracy: 0.5021
Epoch 7/10
120/120 [==============================] - 1s 9ms/step - loss: 0.7737 - accuracy: 0.7341 - val_loss: 2.6438 - val_accuracy: 0.3406
Epoc

In [9]:
temp={} # store the performance


score = model_customized.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test error:", 1-score[1])

temp["Customized Model Test error"]=1-score[1]

import pandas as pd

performance =pd.DataFrame([temp])



Test loss: 1.775053858757019
Test error: 0.5329999923706055
